In [1]:
import numpy as np
import helpers_and_variables as hlp
import ML_helpers as ml_hlp
import shap
import random

import pandas as pd
# import pixiedust
# %%pixie_debuggerR

In [2]:
"""
read the row data as pd data frame
"""
dataPath = "C:/Users/a7mad/Desktop/MEX/PekLung/PekLUng_20210503sav.xlsx"
password = "d46cf574-84e1-11ec-a8a3-0242ac120002" #
rowdataDF = hlp.get_exL_df(stringPath=dataPath, password=password, sheetNum=1)
""" 
read the data info as pd data frame
"""
dataInfoPath = "C:/Users/a7mad/Desktop/MEX/PekLung/Datainformation_minroAdjusted.xlsx"
dataInfoDF = hlp.get_cleaned_dataInfo_df(dataInfoPath)
katInfoDF = hlp.get_cleaned_katInfo_df(dataInfoPath)
"""
get dictionary of data information from data info data fram
"""
dict_of_katInfo = hlp.get_dict_of_katInfoDF(katInfoDF)
dict_of_dataInfo = hlp.get_dict_of_dataInfoDF(dataInfoDF)
""" 
Get the labels, under name Lungcancer_Num
check whether labels are 1=yes LC or 2=No LC 
and get patient with other labels (missing or other diagnoses) removed, total 25 patients
"""
rowdataDF, labels, removed_indices = hlp.get_labels_and_indicesOfUnlabeledPatients(rowdataDF)
""" 
Get the labels, under name Lungcancer_Num
check whether labels are 1=yes LC or 2=No LC 
and get patient with other labels (missing or other diagnoses) removed, total 25 patients
"""
rowdataDF, labels, removed_indices = hlp.get_labels_and_indicesOfUnlabeledPatients(rowdataDF)
"""
Remove the first features, which includes information about the label, like diagnos2, aslo modules names
DiagnosticInvestigation (need to be discussed, since it includes dignostic which means non early prediction)
"""
rowdataDF =  hlp.get_cleanedCol_rowData_df(rowdataDF)



In [ ]:
copy_rowdata = pd.DataFrame.copy(rowdataDF)
copy_rowdata = hlp.get_dates_in_days(copy_rowdata)
# copy_rowdata

In [4]:
# list_of_vars = katInfoDF['Variable'].tolist()
# list_of_labels = katInfoDF['Label'].tolist()
# list_of_values = katInfoDF['Value'].tolist()

# list_of_questions = dataInfoDF['Label'].tolist()
# list_of_quesNams = dataInfoDF['Variable'].tolist()

# cleanedListOfStrings = hlp.get_cleaned_list_of_strings(list_of_questions, stemm=True)
# cleanedListOfStrings
# cleanedWordsArray = hlp.get_array_of_words(cleanedListOfStrings)
"""
get a list with qustions and answers for one patient
"""
# list_ques_answers = list(main_dict['1001'].values())
# list_cleaned_ques_answers = hlp.get_cleaned_list_of_strings(list_ques_answers,stemm=False)
# token_words_array = hlp.get_array_of_words(list_cleaned_ques_answers)
# unique_token_words = np.unique(token_words_array, return_counts=True)
# stemmed_by_nltk = hlp.get_stemmed_strings_as_nltk_SnowballStemmer(list_of_questions, ignore_stopwords=False)
# tokenized_by_nltk =  hlp.get_tokenized_strings_by_nltk(cleanedListOfStrings)

'\nget a list with qustions and answers for one patient\n'

In [5]:
# cols_with_dates = hlp.get_cols_with_dates(copy_rowdata)
# df_dates_before = pd.DataFrame(columns=cols_with_dates)
# df_dates_after = pd.DataFrame(columns=cols_with_dates)
# for column in cols_with_dates:
#     df_dates_before[column] = rowdataDF[column]
#     df_dates_after[column] = copy_rowdata[column]

In [6]:
copy_rowdata.to_csv('C:/Users/a7mad/Desktop/MEX/PekLung/dates_adjusted.csv')
copy_rowdata.to_excel('C:/Users/a7mad/Desktop/MEX/PekLung/dates_adjusted_excel.xlsx')
# x = pd.read_csv('dates_adjusted.csv')
# x = pd.read_excel('dates_adjusted_excel.xlsx')
# del cols_with_dates, df_dates_after

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(df_dates_before)
# display(df_dates_before)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):    
#     display(df_dates_after)

In [19]:
"""
get a dictionary of each patient according to this format:
Patient number: {questions name, the questions text, corresponding answers}
If clear_missings_or_Non true, it clears questions (features),
where answeres are missing or the answers is no/missing
"""
main_dict, ind_num_removed_features = hlp.get_dict_of_questions_answers(copy_rowdata, 
                                                                        dataInfoDF, 
                                                                        katInfoDF, 
                                                                        amount_data = None,
                                                                        clear_missings_or_Non=True)
"""
check how many features were removed by get_dict_of_questions_answers()
You can calculate the actual number of features removed by:
len(rowdataDF.loc[ind_of_max_feature_removed]) - len(main_dict[str(int(rowdataDF.loc[ind_of_max_feature_removed][0]))])-1
obs! (-1 at the end to count for patient number which is not a feature).
"""
max_num_feature_removed = 0
ind_of_max_feature_removed = 0

for ind, count in ind_num_removed_features:
    if count > max_num_feature_removed:
        max_num_feature_removed = count
        ind_of_max_feature_removed = ind
print("most number of features removed is: ", max_num_feature_removed)
print("patient index of most features removed is: ", ind_of_max_feature_removed)
print("Patient number: {0}, with remaining number of features: {1}".format(str(int(rowdataDF.loc[ind_of_max_feature_removed][0])), len(main_dict[str(int(rowdataDF.loc[ind_of_max_feature_removed][0]))])))

#main_dict[str(int(rowdataDF.loc[ind_of_max_feature_removed][0]))]

most number of features removed is:  618
patient index of most features removed is:  98
Patient number: 1174, with remaining number of features: 65


In [20]:
len(main_dict)

645

In [22]:
"""
In case you need to write/save the dictionary
"""
hlp.write_dict_as_json_file(main_dict)

js = hlp.read_dict_from_json_file()
js==main_dict

True

In [18]:
# data_list = hlp.get_data_list_from_main_dict(main_dict, stemm=True)
# data_list

In [ ]:
# vectorizer = ml_hlp.get_tfidf_vectorization_model(data_list)
# X_train_vectorized = ml_hlp.get_tfidf_vectorized_data(tfidf_model=vectorizer,
#                                                       to_be_vectorized_data=data_list)
# X_train = ml_hlp.get_csr_matrix(X_train_vectorized)
